In [ ]:
# Cell 1: Setup and imports
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import math

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

KAGGLE = os.path.exists('/kaggle/input')
BASE_PATH = '/kaggle/input/student-life/dataset' if KAGGLE else 'data/studentlife'
OUTPUT_PATH = '/kaggle/working' if KAGGLE else 'reports'
print(f"Environment: {'Kaggle' if KAGGLE else 'Local'}")

In [ ]:
# Cell 2: Data extraction functions

def get_all_student_ids(base_path):
    """Find student IDs with better path detection"""
    student_ids = set()
    
    # Try multiple possible paths
    possible_paths = [
        os.path.join(base_path, 'sensing', 'phonelock'),
        os.path.join(base_path, 'phonelock'),
        base_path
    ]
    
    for sensing_path in possible_paths:
        if os.path.exists(sensing_path):
            print(f"  Checking: {sensing_path}")
            for f in os.listdir(sensing_path):
                if f.startswith('phonelock_u') and f.endswith('.csv'):
                    student_ids.add(f.replace('phonelock_', '').replace('.csv', ''))
            if student_ids:
                break
    
    if not student_ids:
        print(f"⚠️  No students found. Available paths:")
        for p in possible_paths:
            print(f"    {p} - Exists: {os.path.exists(p)}")
        if os.path.exists(base_path):
            print(f"  Contents of {base_path}:")
            for item in os.listdir(base_path)[:10]:
                print(f"    - {item}")
    
    return sorted(list(student_ids))

def extract_sleep_from_phonelock(base_path, student_id):
    phonelock_path = os.path.join(base_path, 'sensing', 'phonelock', f'phonelock_{student_id}.csv')
    if not os.path.exists(phonelock_path): return {}
    try:
        df = pd.read_csv(phonelock_path, encoding='utf-8-sig')
        if 'start' not in df.columns: return {}
        df['start_dt'] = pd.to_datetime(df['start'], unit='s')
        df['duration_hours'] = (df['end'] - df['start']) / 3600
        df['date'] = df['start_dt'].dt.date
        df['start_hour'] = df['start_dt'].dt.hour
        daily_sleep = {}
        for date in df['date'].unique():
            day_data = df[df['date'] == date]
            night = day_data[((day_data['start_hour'] >= 22) | (day_data['start_hour'] <= 10)) & (day_data['duration_hours'] >= 3)]
            if len(night) > 0: daily_sleep[date] = min(night['duration_hours'].max(), 12)
        return daily_sleep
    except: return {}

def extract_activity(base_path, student_id):
    path = os.path.join(base_path, 'sensing', 'activity', f'activity_{student_id}.csv')
    if not os.path.exists(path): return {}, {}
    try:
        df = pd.read_csv(path, encoding='utf-8-sig')
        if 'timestamp' not in df.columns: return {}, {}
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
        df['date'] = df['datetime'].dt.date
        act_col = ' activity inference' if ' activity inference' in df.columns else 'activity_inference'
        if act_col not in df.columns: return {}, {}
        exercise, steps = {}, {}
        for date in df['date'].unique():
            day = df[df['date'] == date]
            active = day[day[act_col].isin([1, 2])]
            mins = len(active) * 3 / 60
            exercise[date] = min(mins, 120)
            steps[date] = int(mins * 100)
        return exercise, steps
    except: return {}, {}

def extract_screen_time(base_path, student_id):
    path = os.path.join(base_path, 'sensing', 'phonelock', f'phonelock_{student_id}.csv')
    if not os.path.exists(path): return {}
    try:
        df = pd.read_csv(path, encoding='utf-8-sig')
        df['start_dt'] = pd.to_datetime(df['start'], unit='s')
        df['date'] = df['start_dt'].dt.date
        df['locked'] = (df['end'] - df['start']) / 3600
        screen = {}
        for date in df['date'].unique():
            day = df[df['date'] == date]
            screen[date] = min(max(0, 18 - day['locked'].sum()), 16)
        return screen
    except: return {}

def extract_social(base_path, student_id):
    path = os.path.join(base_path, 'sensing', 'conversation', f'conversation_{student_id}.csv')
    social = {}
    if os.path.exists(path):
        try:
            df = pd.read_csv(path, encoding='utf-8-sig')
            col = 'start_timestamp' if 'start_timestamp' in df.columns else ' start_timestamp'
            if col in df.columns:
                df['datetime'] = pd.to_datetime(df[col], unit='s')
                df['date'] = df['datetime'].dt.date
                for date in df['date'].unique():
                    social[date] = len(df[df['date'] == date])
        except: pass
    return social

def extract_work(base_path, student_id):
    path = os.path.join(base_path, 'app_usage', f'running_app_{student_id}.csv')
    if not os.path.exists(path): return {}
    try:
        df = pd.read_csv(path, encoding='utf-8-sig')
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
        df['date'] = df['datetime'].dt.date
        pkg_col = 'RUNNING_TASKS_topActivity_mPackage'
        if pkg_col not in df.columns: return {}
        work_pkgs = ['com.google', 'edu.', 'blackboard', 'chrome', 'microsoft']
        work = {}
        for date in df['date'].unique():
            day = df[df['date'] == date]
            mask = day[pkg_col].astype(str).apply(lambda x: any(p in x.lower() for p in work_pkgs))
            work[date] = min(len(day[mask]) * 10 / 3600, 14)
        return work
    except: return {}

print("Data extraction functions ready")

In [ ]:
# Cell 3: Build dataset with smart imputation

def build_dataset(base_path):
    student_ids = get_all_student_ids(base_path)
    print(f"Found {len(student_ids)} students")
    
    if len(student_ids) == 0:
        print("❌ No students found - cannot proceed")
        return pd.DataFrame()
    
    records = []
    for i, sid in enumerate(student_ids):
        sleep = extract_sleep_from_phonelock(base_path, sid)
        exercise, steps = extract_activity(base_path, sid)
        screen = extract_screen_time(base_path, sid)
        social = extract_social(base_path, sid)
        work = extract_work(base_path, sid)
        
        all_dates = set()
        for d in [sleep, exercise, screen, social, work]: all_dates.update(d.keys())
        
        for date in sorted(all_dates):
            records.append({
                'student_id': sid, 'date': date,
                'sleep_hours': sleep.get(date, np.nan),
                'exercise_minutes': exercise.get(date, np.nan),
                'steps_count': steps.get(date, np.nan),
                'screen_time_hours': screen.get(date, np.nan),
                'social_interactions': social.get(date, np.nan),
                'work_hours': work.get(date, np.nan),
            })
        if (i+1) % 10 == 0: print(f"  Processed {i+1}/{len(student_ids)}")
    
    if not records:
        print("⚠️  No records extracted")
        return pd.DataFrame()
    
    df = pd.DataFrame(records)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(['student_id', 'date']).reset_index(drop=True)
    print(f"\n✓ Dataset: {len(df)} records, {df['student_id'].nunique()} students")
    
    # Report missing data before imputation
    missing_pct = (df.isnull().sum() / len(df) * 100).round(1)
    print(f"\n📊 Missing data:")
    for col, pct in missing_pct.items():
        if pct > 0 and col not in ['student_id', 'date']:
            print(f"  {col}: {pct}%")
    
    return df

df = build_dataset(BASE_PATH)
if len(df) > 0:
    df.head()
else:
    print("⚠️  Empty dataset - check paths above")

In [ ]:
# Cell 4: Smart missing data imputation + sequence creation

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer

def smart_impute_student_data(df, feature_cols):
    """Multi-strategy imputation: temporal patterns + KNN + iterative modeling"""
    
    print("\n🧠 Smart imputation pipeline:")
    df_imputed = df.copy()
    
    # Strategy 1: Forward/Backward fill per student (temporal continuity)
    print("  1. Temporal fill (forward/backward per student)...")
    for sid in df_imputed['student_id'].unique():
        mask = df_imputed['student_id'] == sid
        for col in feature_cols:
            df_imputed.loc[mask, col] = df_imputed.loc[mask, col].fillna(method='ffill').fillna(method='bfill')
    
    still_missing = df_imputed[feature_cols].isnull().sum().sum()
    print(f"     → {still_missing} values still missing")
    
    # Strategy 2: Rolling mean (7-day window per student)
    if still_missing > 0:
        print("  2. Rolling mean imputation (7-day window)...")
        for sid in df_imputed['student_id'].unique():
            mask = df_imputed['student_id'] == sid
            sdata = df_imputed[mask].copy()
            for col in feature_cols:
                rolling_mean = sdata[col].rolling(window=7, min_periods=1, center=True).mean()
                df_imputed.loc[mask, col] = df_imputed.loc[mask, col].fillna(rolling_mean)
        
        still_missing = df_imputed[feature_cols].isnull().sum().sum()
        print(f"     → {still_missing} values still missing")
    
    # Strategy 3: KNN Imputation (find similar days from other students)
    if still_missing > 0:
        print("  3. KNN imputation (k=5, similar days across students)...")
        knn_imputer = KNNImputer(n_neighbors=5, weights='distance')
        df_imputed[feature_cols] = knn_imputer.fit_transform(df_imputed[feature_cols])
        
        still_missing = df_imputed[feature_cols].isnull().sum().sum()
        print(f"     → {still_missing} values still missing")
    
    # Strategy 4: Iterative Imputation (predict missing using Random Forest)
    if still_missing > 0:
        print("  4. Iterative imputation (Random Forest predictor)...")
        iter_imputer = IterativeImputer(max_iter=10, random_state=42, verbose=0)
        df_imputed[feature_cols] = iter_imputer.fit_transform(df_imputed[feature_cols])
        
        still_missing = df_imputed[feature_cols].isnull().sum().sum()
        print(f"     → {still_missing} values remaining")
    
    # Strategy 5: Global median fallback (last resort)
    if still_missing > 0:
        print("  5. Global median fallback (last resort)...")
        for col in feature_cols:
            df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())
    
    # Clip to realistic ranges
    df_imputed['sleep_hours'] = df_imputed['sleep_hours'].clip(2, 14)
    df_imputed['exercise_minutes'] = df_imputed['exercise_minutes'].clip(0, 180)
    df_imputed['steps_count'] = df_imputed['steps_count'].clip(0, 30000)
    df_imputed['screen_time_hours'] = df_imputed['screen_time_hours'].clip(0, 18)
    df_imputed['social_interactions'] = df_imputed['social_interactions'].clip(0, 50)
    df_imputed['work_hours'] = df_imputed['work_hours'].clip(0, 16)
    
    print(f"✓ Imputation complete - 0 missing values")
    return df_imputed

def create_sequences(df, feature_cols, seq_length=7):
    """Create sequences from imputed data"""
    sequences, targets, sids, dates = [], [], [], []
    
    # Smart imputation first
    df_clean = smart_impute_student_data(df, feature_cols)
    
    for sid in df_clean['student_id'].unique():
        sdata = df_clean[df_clean['student_id'] == sid].sort_values('date').copy()
        if len(sdata) < seq_length + 1: continue
        
        vals = sdata[feature_cols].values
        dvals = sdata['date'].values
        
        for i in range(len(vals) - seq_length):
            sequences.append(vals[i:i+seq_length])
            targets.append(vals[i+seq_length])
            sids.append(sid)
            dates.append(dvals[i+seq_length])
    
    X, y = np.array(sequences), np.array(targets)
    print(f"\n✓ Created {len(X)} sequences from {df_clean['student_id'].nunique()} students")
    return X, y, sids, dates

feature_cols = ['sleep_hours', 'exercise_minutes', 'steps_count', 
               'screen_time_hours', 'social_interactions', 'work_hours']

if len(df) > 0:
    X, y, student_ids, dates = create_sequences(df, feature_cols)
    print(f"X shape: {X.shape}, y shape: {y.shape}")
else:
    print("⚠️  Skipping sequence creation - no data available")

In [ ]:
# Cell 5: Model definitions

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.name = "LSTM"
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, 
                           dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, input_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class BiLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.name = "BiLSTM"
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, 
                           bidirectional=True, dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim * 2, input_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class GRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.name = "GRU"
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True,
                         dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, input_dim)
    def forward(self, x):
        out, _ = self.gru(x)
        return self.fc(out[:, -1, :])

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(1, max_len, d_model)
        position = torch.arange(max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[0, :, 0::2] = torch.sin(position * div_term)
        pe[0, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1)])

class TransformerModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, nhead=4, dropout=0.3):
        super().__init__()
        self.name = "Transformer"
        self.input_proj = nn.Linear(input_dim, hidden_dim)
        self.pos_encoder = PositionalEncoding(hidden_dim, dropout)
        layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=nhead, 
                                          dim_feedforward=hidden_dim*4, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(layer, num_layers)
        self.fc = nn.Linear(hidden_dim, input_dim)
    def forward(self, x):
        x = self.pos_encoder(self.input_proj(x))
        return self.fc(self.transformer(x)[:, -1, :])

class CNNLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.3):
        super().__init__()
        self.name = "CNN-LSTM"
        self.conv1 = nn.Conv1d(input_dim, hidden_dim, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True,
                           dropout=dropout if num_layers > 1 else 0)
        self.fc = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.dropout(self.relu(self.conv1(x)))
        x = self.relu(self.conv2(x)).permute(0, 2, 1)
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

class MLPModel(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, seq_length=7, dropout=0.3):
        super().__init__()
        self.name = "MLP"
        self.fc1 = nn.Linear(input_dim * seq_length, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x.view(x.size(0), -1))))
        return self.fc3(self.dropout(self.relu(self.fc2(x))))

print("Model definitions ready: LSTM, BiLSTM, GRU, Transformer, CNN-LSTM, MLP")

In [ ]:
# Cell 6: Training utilities

def train_model(model, train_loader, val_loader, epochs=50, lr=0.001, patience=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    
    best_val_loss = float('inf')
    patience_counter = 0
    best_state = None
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            loss = criterion(model(X_batch), y_batch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                val_loss += criterion(model(X_batch.to(device)), y_batch.to(device)).item()
        val_loss /= len(val_loader)
        scheduler.step(val_loss)
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience: break
    
    if best_state: model.load_state_dict(best_state)
    return model, best_val_loss

def run_cv(model_class, model_kwargs, X, y, n_folds=5, epochs=50):
    kfold = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    results = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Normalize
        X_mean, X_std = X_train.mean(axis=(0,1)), X_train.std(axis=(0,1)) + 1e-8
        y_mean, y_std = y_train.mean(axis=0), y_train.std(axis=0) + 1e-8
        
        X_train_n = (X_train - X_mean) / X_std
        X_val_n = (X_val - X_mean) / X_std
        y_train_n = (y_train - y_mean) / y_std
        
        train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_train_n), 
                                                torch.FloatTensor(y_train_n)), batch_size=32, shuffle=True)
        val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_val_n), 
                                              torch.FloatTensor((y_val - y_mean) / y_std)), batch_size=32)
        
        model = model_class(**model_kwargs)
        model, _ = train_model(model, train_loader, val_loader, epochs=epochs)
        
        model.eval()
        with torch.no_grad():
            device = next(model.parameters()).device
            preds_n = model(torch.FloatTensor(X_val_n).to(device)).cpu().numpy()
        preds = preds_n * y_std + y_mean
        
        results.append({
            'mae': mean_absolute_error(y_val, preds),
            'rmse': np.sqrt(mean_squared_error(y_val, preds)),
            'r2': r2_score(y_val, preds)
        })
    
    return {
        'mae_mean': np.mean([r['mae'] for r in results]),
        'mae_std': np.std([r['mae'] for r in results]),
        'rmse_mean': np.mean([r['rmse'] for r in results]),
        'r2_mean': np.mean([r['r2'] for r in results]),
        'r2_std': np.std([r['r2'] for r in results]),
    }

print("Training utilities ready")

In [ ]:
# Cell 7: RUN MODEL COMPARISON EXPERIMENT

input_dim = len(feature_cols)
hidden_dim = 64

models = [
    ("LSTM", LSTMModel, {'input_dim': input_dim, 'hidden_dim': hidden_dim}),
    ("BiLSTM", BiLSTMModel, {'input_dim': input_dim, 'hidden_dim': hidden_dim}),
    ("GRU", GRUModel, {'input_dim': input_dim, 'hidden_dim': hidden_dim}),
    ("Transformer", TransformerModel, {'input_dim': input_dim, 'hidden_dim': hidden_dim}),
    ("CNN-LSTM", CNNLSTMModel, {'input_dim': input_dim, 'hidden_dim': hidden_dim}),
    ("MLP", MLPModel, {'input_dim': input_dim, 'hidden_dim': 128, 'seq_length': 7}),
]

print("="*70)
print("MODEL COMPARISON (5-fold CV)")
print("="*70)

results = {}
for name, model_class, kwargs in models:
    print(f"\nTesting: {name}")
    try:
        res = run_cv(model_class, kwargs, X, y, n_folds=5, epochs=50)
        results[name] = res
        print(f"  R²: {res['r2_mean']:.4f} ± {res['r2_std']:.4f}")
        print(f"  MAE: {res['mae_mean']:.4f}")
    except Exception as e:
        print(f"  ❌ Error: {e}")

# Rank by R²
ranking = sorted(results.items(), key=lambda x: x[1]['r2_mean'], reverse=True)

print("\n" + "="*70)
print("🏆 LEADERBOARD")
print("="*70)
for i, (name, res) in enumerate(ranking, 1):
    medal = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "  "
    print(f"{medal} {i}. {name:<15} R²={res['r2_mean']:.4f}  MAE={res['mae_mean']:.4f}")

best_name = ranking[0][0]
print(f"\n✅ BEST MODEL: {best_name}")

In [ ]:
# Cell 8: Train and save best model

# Get best model class
best_class, best_kwargs = None, None
for name, cls, kwargs in models:
    if name == best_name:
        best_class, best_kwargs = cls, kwargs
        break

# Normalize all data
X_mean = X.mean(axis=(0, 1))
X_std = X.std(axis=(0, 1)) + 1e-8
y_mean = y.mean(axis=0)
y_std = y.std(axis=0) + 1e-8

X_norm = (X - X_mean) / X_std
y_norm = (y - y_mean) / y_std

# 90/10 split
n = int(0.9 * len(X))
train_loader = DataLoader(TensorDataset(torch.FloatTensor(X_norm[:n]), 
                                        torch.FloatTensor(y_norm[:n])), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.FloatTensor(X_norm[n:]), 
                                      torch.FloatTensor(y_norm[n:])), batch_size=32)

print(f"Training final {best_name} model...")
final_model = best_class(**best_kwargs)
final_model, _ = train_model(final_model, train_loader, val_loader, epochs=100)

# Save checkpoint
checkpoint = {
    'model_name': best_name,
    'model_state': final_model.state_dict(),
    'model_kwargs': best_kwargs,
    'feature_cols': feature_cols,
    'scaler_mean_X': X_mean.tolist(),
    'scaler_std_X': X_std.tolist(),
    'scaler_mean_y': y_mean.tolist(),
    'scaler_std_y': y_std.tolist(),
    'cv_results': results[best_name],
    'all_results': {k: {kk: vv for kk, vv in v.items()} for k, v in results.items()},
    'created_at': datetime.now().isoformat()
}

torch.save(checkpoint, f'{OUTPUT_PATH}/best_behavioral_model.pt')
print(f"\n✓ Saved: best_behavioral_model.pt")
print(f"  Model: {best_name}")
print(f"  R²: {results[best_name]['r2_mean']:.4f}")

In [ ]:
# Cell 9: Save results JSON

results_json = {
    'experiment_date': datetime.now().isoformat(),
    'dataset': {
        'total_sequences': len(X),
        'num_students': len(set(student_ids)),
        'features': feature_cols
    },
    'models_tested': list(results.keys()),
    'results': {k: v for k, v in results.items()},
    'ranking': [(name, res['r2_mean']) for name, res in ranking],
    'best_model': {
        'name': best_name,
        'r2': results[best_name]['r2_mean'],
        'mae': results[best_name]['mae_mean']
    }
}

with open(f'{OUTPUT_PATH}/model_comparison_results.json', 'w') as f:
    json.dump(results_json, f, indent=2)

print("✓ Saved: model_comparison_results.json")
print("\n" + "="*70)
print("EXPERIMENT COMPLETE!")
print("="*70)
print("\nDownload:")
print("  1. best_behavioral_model.pt")
print("  2. model_comparison_results.json")
print("\nNext: Combine with synthetic model for two-stage pipeline!")